<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: Юрий Исаков и Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 4. Линейные модели классификации и регрессии
## <center>  Практика. Идентификация пользователя с помощью логистической регрессии

Тут мы воспроизведем парочку бенчмарков нашего соревнования и вдохновимся побить третий бенчмарк, а также остальных участников. Веб-формы для отправки ответов тут не будет, ориентир – [leaderboard](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard) соревнования.

In [2]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
sklearn.__version__

'0.19.0'

### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [3]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('../../data/train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('../../data/test_sessions.csv',
                      index_col='session_id')

# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

In [4]:
# посмотрим на заголовок обучающей выборки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [5]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# загрузим словарик сайтов
with open(r"../../data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всего сайтов:', sites_dict_df.shape[0])
sites_dict_df.head()

всего сайтов: 48371


,site
1418,sciences-technologies.lefigaro.fr
14576,mtbwal.blogspot.com
5466,mj3j3nqk0n.b.ad6media.fr
21368,www.clermontechecs.net
7960,www.maitianquan.com


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [6]:
y_train = train_df['target']

Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [8]:
def convert_sites_to_sequence(row):
    return ' '.join([str(index) for index in row.values if index != 0])

In [9]:
vectorizer = TfidfVectorizer()
corpus = train_df.drop('target', axis=1)[sites].apply(convert_sites_to_sequence, axis=1).values
X_train_sparse = vectorizer.fit_transform(corpus)

In [10]:
corpus = test_df[sites].apply(convert_sites_to_sequence, axis=1).values
X_test_sparse = vectorizer.transform(corpus)

In [11]:
X_train_sparse.shape, X_test_sparse.shape

((253561, 41592), (82797, 41592))

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation). 

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

In [53]:
from sklearn.model_selection import train_test_split

def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=(1 - ratio))
    est = LogisticRegression(C=C, random_state=seed)
    est.fit(X_train, y_train)
    return roc_auc_score(y_test, est.predict_proba(X_test)[:, 1])

**Посмотрите, какой получился ROC AUC на отложенной выборке.**

In [13]:
get_auc_lr_valid(X_train_sparse, y_train)

0.92357498288186135

Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [14]:
# функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

In [15]:
new_train = pd.DataFrame(index=train_df.index)
new_test = pd.DataFrame(index=test_df.index)

In [16]:
def scale_feature(train, test, feat_name):
    scaler = StandardScaler()
    scaler.fit(train[feat_name].astype('float64').values.reshape(-1, 1))
    train[feat_name + '_s'] = scaler.transform(train[feat_name].astype('float64').values.reshape(-1, 1))
    test[feat_name + '_s'] = scaler.transform(test[feat_name].astype('float64').values.reshape(-1, 1))    

### Месяц начала сессии

In [17]:
def transform_sess_start(ts):
    return ts.month

new_train['year_month'] = train_df['time1'].apply(transform_sess_start)
new_test['year_month'] = test_df['time1'].apply(transform_sess_start)

In [18]:
scale_feature(new_train, new_test, 'year_month')

### Время начала сессии и утро ли

In [19]:
def get_hour(ts):
    return ts.hour

def get_morning(ts):
    if ts.hour <= 11:
        return 1
    else:
        return 0
    
new_train['start_hour'] = train_df['time1'].apply(get_hour)
new_test['start_hour'] = test_df['time1'].apply(get_hour)
new_train['morning'] = train_df['time1'].apply(get_morning)
new_test['morning'] = test_df['time1'].apply(get_morning)

In [20]:
scale_feature(new_train, new_test, 'start_hour')

### Продолжительность сессии в секундах

In [21]:
def get_sess_length(row):
    dropped_row = row.dropna()
    length_ts = dropped_row.iloc[-1] - dropped_row.iloc[-(2%len(dropped_row))]
    return length_ts/np.timedelta64(1, 's')

new_train['sess_length'] = train_df[times].apply(get_sess_length, axis=1)
new_test['sess_length'] = test_df[times].apply(get_sess_length, axis=1)

In [22]:
scale_feature(new_train, new_test, 'sess_length')

### Количество посещенных сайтов

In [23]:
def get_site_count(row):
    dropped_row = row.dropna()
    return len(dropped_row)

new_train['site_count'] = train_df[times].apply(get_site_count, axis=1)
new_test['site_count'] = test_df[times].apply(get_site_count, axis=1)

In [24]:
scale_feature(new_train, new_test, 'site_count')

### Сессия больше 5 минут

In [25]:
def get_more5min(val):
    if val/60 > 5:
        return 1
    else:
        return 0

new_train['more5min'] = new_train['sess_length'].apply(get_more5min)
new_test['more5min'] = new_test['sess_length'].apply(get_more5min)

### Вечер ли(час >18)

In [47]:
def get_evening(ts):
    if ts.hour >= 20:
        return 1
    else:
        return 0
    
new_train['evening'] = train_df['time1'].apply(get_evening)
new_test['evening'] = test_df['time1'].apply(get_evening)

In [48]:
new_test.head()

,year_month,year_month_s,start_hour,morning,start_hour_s,sess_length,sess_length_s,site_count,site_count_s,more5min,evening
session_id,,,,,,,,,,,
1,10,1.474950,11,1,-0.407823,0.0,-0.203780,10,0.278784,0,0
2,7,0.668778,11,1,-0.407823,23.0,0.029935,10,0.278784,0,0
3,12,2.012399,15,0,0.858234,3.0,-0.173295,10,0.278784,0,0
4,11,1.743675,10,1,-0.724338,0.0,-0.203780,10,0.278784,0,0
5,5,0.131329,15,0,0.858234,4.0,-0.163134,10,0.278784,0,0


In [49]:
columns = ['year_month_s', 'start_hour_s', 'morning', 'sess_length_s', 'evening']

In [50]:
X_train = csr_matrix(hstack([X_train_sparse, new_train[columns].values]))
X_test = csr_matrix(hstack([X_test_sparse, new_test[columns].values]))

### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитайте качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию `C=1`:

In [82]:
get_auc_lr_valid(X_train, y_train, C=21.544346900318821)

0.98244416132499901

Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален. 

In [56]:
for C in np.logspace(-3, 1, 10):
    print('C: ', C, 'ROC_AUC: ', get_auc_lr_valid(X_train, y_train, C=C))

C:  0.001 ROC_AUC:  0.851703300949
C:  0.00278255940221 ROC_AUC:  0.881986225218
C:  0.00774263682681 ROC_AUC:  0.916745730006
C:  0.0215443469003 ROC_AUC:  0.935669482133
C:  0.0599484250319 ROC_AUC:  0.938815323905
C:  0.16681005372 ROC_AUC:  0.957114797271
C:  0.464158883361 ROC_AUC:  0.963487720432
C:  1.29154966501 ROC_AUC:  0.975476574638
C:  3.5938136638 ROC_AUC:  0.981603485323
C:  10.0 ROC_AUC:  0.9844390911


In [80]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

cv = ShuffleSplit(n_splits=2, test_size=0.1)
est = LogisticRegression()
params = {
    'C': np.logspace(-3, 10, 10)
}
grid = GridSearchCV(est, params, scoring='roc_auc', n_jobs=-1)
grid = grid.fit(X_train, y_train)

In [81]:
for score, params in  zip(grid.cv_results_['mean_test_score'], grid.cv_results_['params']):
    print(score, params)

0.693765565768 {'C': 0.001}
0.842109892311 {'C': 0.027825594022071243}
0.89884951913 {'C': 0.774263682681127}
0.911338878034 {'C': 21.544346900318821}
0.906781701343 {'C': 599.48425031894089}
0.893646174313 {'C': 16681.005372000593}
0.885549555995 {'C': 464158.88336127723}
0.88246265063 {'C': 12915496.650148828}
0.881879335204 {'C': 359381366.38046259}
0.886443940835 {'C': 10000000000.0}


In [83]:
y_pred = grid.best_estimator_.predict_proba(X_test)[:, 1]
write_to_submission_file(y_pred, './../../data/alice_result3.csv')